<a href="https://colab.research.google.com/github/JooEHyeon/SaltLux_Chatbot/blob/master/apparel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import re
import pandas as pd
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
wear_data = pd.read_csv("./drive/My Drive/wear.csv")
print(wear_data.shape)
customer = wear_data[wear_data.SPEAKER == "고객"].SENTENCE
store = wear_data[wear_data.SPEAKER == "점원"].SENTENCE
print(customer.shape, store.shape) # 질문의 개수와 답의 개수가 일치하지 않는다.
store.head()
print(type(wear_data))

(15826, 20)
(8381,) (7445,)
<class 'pandas.core.frame.DataFrame'>


In [ ]:
prev = "고객"
store_arr = []
customer_arr = []
store_stc = ""
customer_stc = ""

for i in range(wear_data.shape[0]):
    if (prev == wear_data.iloc[i].SPEAKER):
        if prev == "점원":
             store_stc += (" "+wear_data.iloc[i].SENTENCE)
        else : 
            customer_stc += (" "+wear_data.iloc[i].SENTENCE)
            
    elif prev == "점원":
        store_arr.append(store_stc)
        customer_stc = wear_data.iloc[i].SENTENCE
        prev = "고객"
    else :
        customer_arr.append(customer_stc)
        store_stc = wear_data.iloc[i].SENTENCE
        prev = "점원"

print(len(store_arr))
print(len(customer_arr))
print(store_arr[-1])
print(customer_arr[-1]) # 자료 상에서 이후에는 계속 고객의 물음만 계속된다. 코드 레벨에서 이 부분은 빼게 구현했다. (stc는 만들어지지만 arr에 append 안하게 된다.)

7301
7301
요즘 파스텔 톤이 유행이에요
요즘 유행하는 색깔이 뭐예요?


In [ ]:
snc_temp = {'customer' : customer_arr, 'store' : store_arr}
snc = pd.DataFrame(snc_temp)
snc

,customer,store
0,신발은 여기 있는 게 다예요?,네 성인이나 아동 다 있어요 발 사이즈 몇 신으세요?
1,230이요,편하게 신을 수 있는 거 찾으세요?
2,네 봄이니까 편하게 신을 수 있는 거,이런 건 어떠세요? 이런 거도 신발 무척 편하거든요
3,굽 좀 높은 거 없나요?,봄 상품은 아직 어른 제품이 많이 안나왔습니다
4,언제 들어와요?,이번주 지나면 들어올 거예요
...,...,...
7296,블라우스나 티셔츠 같은거는 얼마부터 있어요?,대부분 블라우스는 팔만원이나 십만원대로 있어요
7297,이런 스카프는 얼마예요?,그 스카프는 팔만 오천원이요
7298,이런 옷은 얼마예요?,그 옷은 십이만 팔천 원이에요
7299,지금 봄옷은 이르지 않나요?,지금 봄옷 입기 괜찮아요


In [ ]:
encoder_input, decoder_input, decoder_output = [], [], []

for stc in snc['customer']:
    encoder_input.append(stc.split())

for stc in snc['store']:
    decoder_input.append(("<start> "+stc).split())
    decoder_output.append((stc + " <end>").split())

print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_output[:3])

[['신발은', '여기', '있는', '게', '다예요?'], ['230이요'], ['네', '봄이니까', '편하게', '신을', '수', '있는', '거']]
[['<start>', '네', '성인이나', '아동', '다', '있어요', '발', '사이즈', '몇', '신으세요?'], ['<start>', '편하게', '신을', '수', '있는', '거', '찾으세요?'], ['<start>', '이런', '건', '어떠세요?', '이런', '거도', '신발', '무척', '편하거든요']]
[['네', '성인이나', '아동', '다', '있어요', '발', '사이즈', '몇', '신으세요?', '<end>'], ['편하게', '신을', '수', '있는', '거', '찾으세요?', '<end>'], ['이런', '건', '어떠세요?', '이런', '거도', '신발', '무척', '편하거든요', '<end>']]


In [ ]:
tokenizer_quest = Tokenizer()
tokenizer_quest.fit_on_texts(encoder_input)
encoder_input = tokenizer_quest.texts_to_sequences(encoder_input)

# 1~(패딩하기 전) -> 0~(패딩하고 난 뒤)
tokenizer_ans = Tokenizer()
tokenizer_ans.fit_on_texts(decoder_input)
tokenizer_ans.fit_on_texts(decoder_output) # decoder_input, decoder_output이 다 똑같은데 start, end만 다르다. 이 작업을 통해 end에도 정수 인코딩을 할 수 있는 tokenizer_en이 생성된다.
# 정수 인코딩할 때는 1부터 인덱스를 부여
decoder_input = tokenizer_ans.texts_to_sequences(decoder_input)
decoder_output = tokenizer_ans.texts_to_sequences(decoder_output)

# 이제 encoder_input, decoder_input, decoder_output 에는 정수 인코딩 완료
print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_output[:3])

[[64, 18, 26, 41, 323], [907], [10, 2500, 142, 90, 7, 26, 8]]
[[1, 3, 2633, 2634, 9, 5, 196, 31, 78, 145], [1, 218, 905, 39, 32, 18, 51], [1, 10, 66, 21, 10, 425, 104, 1576, 1148]]
[[3, 2633, 2634, 9, 5, 196, 31, 78, 145, 2], [218, 905, 39, 32, 18, 51, 2], [10, 66, 21, 10, 425, 104, 1576, 1148, 2]]


In [ ]:
encoder_input = pad_sequences(encoder_input, padding="post") # LSTM 을 쓸 것이므로 뒤로 갈수록 정보가 손실되기에 pre 방식(디폴트) 로 패딩해주었다. 
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_output = pad_sequences(decoder_output, padding="post")

print(encoder_input.shape)
print(decoder_input.shape)


(7301, 16)
(7301, 22)


In [ ]:
# 나중에 prediction 할때 사용하기 위함 (인덱스로 단어 찾기)
ans_to_index = tokenizer_ans.word_index # 단어로 인덱스 찾기
index_to_ans = tokenizer_ans.index_word # 인덱스로 단어 찾기
# index_to_ans[0] = ""
print(ans_to_index)
print(index_to_ans)

{'<start>': 1, '<end>': 2, '네': 3, '이': 4, '있어요': 5, '그': 6, '있습니다': 7, '원이에요': 8, '다': 9, '이런': 10, '네,': 11, '잘': 12, '없어요': 13, '지금': 14, '돼요': 15, '한': 16, '많이': 17, '거': 18, '원이요': 19, '이거는': 20, '어떠세요?': 21, '여기': 22, '안': 23, '옷은': 24, '제품은': 25, '거예요': 26, '더': 27, '어떻게': 28, '사이즈가': 29, '어떤': 30, '사이즈': 31, '있는': 32, '원입니다': 33, '가방은': 34, '됩니다': 35, '오천': 36, '그럼': 37, '좀': 38, '수': 39, '이게': 40, '아니요': 41, '이거': 42, '거는': 43, '가능합니다': 44, '만': 45, '고객님': 46, '이만': 47, '해요': 48, '두': 49, '나가요': 50, '찾으세요?': 51, '이건': 52, '제일': 53, '이쪽에': 54, '구천': 55, '삼만': 56, '그거는': 57, '저': 58, '잠시만': 59, '번': 60, '맞아요': 61, '정도': 62, '옷': 63, '원': 64, '제품': 65, '건': 66, '좋아요': 67, '사이즈는': 68, '게': 69, '3만': 70, '세일해서': 71, '있고': 72, '세일': 73, '제품이': 74, '9천': 75, '다른': 76, '드릴게요': 77, '몇': 78, '5천': 79, '따로': 80, '사만': 81, '신어보세요': 82, '되세요?': 83, '나와요': 84, '나온': 85, '입니다': 86, '괜찮아요': 87, '요즘': 88, '알겠습니다': 89, '것': 90, '않아요': 91, '조금': 92, '팔천': 93, '안에': 94, '이쪽으로': 95, '오만': 96, '같아요

In [ ]:
test_size = 1800
encoder_input_train = encoder_input[:-test_size]
decoder_input_train = decoder_input[:-test_size]
decoder_output_train = decoder_output[:-test_size]

encoder_input_test = encoder_input[-test_size:]
decoder_input_test = decoder_input[-test_size:]
decoder_output_test = decoder_output[-test_size:]

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [ ]:
encoder_inputs = Input(shape=(16,))
encoder_embed = Embedding(len(tokenizer_quest.word_index)+1, 50)(encoder_inputs) # encoder_inputs 을 입력으로 50 차원으로 Embedding
encoder_mask = Masking(mask_value=0)(encoder_embed)
encoder_outputs, h_state, c_state = LSTM(50, return_state=True)(encoder_mask) # 아까 50 차원으로 맞춰줬기에 첫 인자로 50 을 넣어준다.

In [ ]:
decoder_inputs = Input(shape=(22,))
decoder_embed = Embedding(len(tokenizer_ans.word_index)+1, 50)(decoder_inputs)
decoder_mask = Masking(mask_value=0)(decoder_embed)
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_mask, initial_state=[h_state, c_state])
decoder_dense = Dense(len(tokenizer_ans.word_index)+1, activation='softmax')
decoder_softmax_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
# sparse는 라벨이 정수 형태로 제공될 때 사용되는 함수 (그냥 categorical은 원핫 벡터로 라벨이 제공될 때)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
# 레이어 별로 가중치가 학습되는 것임
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_output_train, validation_data = ([encoder_input_test, decoder_input_test], decoder_output_test), batch_size = 128, epochs = 20)

Epoch 1/20
43/43 [==============================] - 47s 1s/step - loss: 1.3819 - acc: 0.8305 - val_loss: 1.5798 - val_acc: 0.8165
Epoch 2/20
43/43 [==============================] - 45s 1s/step - loss: 1.3532 - acc: 0.8304 - val_loss: 1.5914 - val_acc: 0.8168
Epoch 3/20
43/43 [==============================] - 45s 1s/step - loss: 1.3368 - acc: 0.8304 - val_loss: 1.5954 - val_acc: 0.8148
Epoch 4/20
43/43 [==============================] - 45s 1s/step - loss: 1.3231 - acc: 0.8304 - val_loss: 1.6006 - val_acc: 0.8155
Epoch 5/20
43/43 [==============================] - 45s 1s/step - loss: 1.3109 - acc: 0.8305 - val_loss: 1.6097 - val_acc: 0.8154
Epoch 6/20
43/43 [==============================] - 45s 1s/step - loss: 1.2998 - acc: 0.8305 - val_loss: 1.6164 - val_acc: 0.8162
Epoch 7/20
43/43 [==============================] - 45s 1s/step - loss: 1.2896 - acc: 0.8306 - val_loss: 1.6207 - val_acc: 0.8157
Epoch 8/20
43/43 [==============================] - 45s 1s/step - loss: 1.2793 - acc: 0.83

# Prediction

In [ ]:
encoder_model = Model(encoder_inputs, [h_state, c_state])
encoder_h_state = Input(shape=(50,))
encoder_c_state = Input(shape=(50,))
pd_decoder_outputs, pd_h_state, pd_c_state = decoder_lstm(decoder_mask, initial_state=[encoder_h_state, encoder_c_state])
pd_decoder_softmax_outputs = decoder_dense(pd_decoder_outputs)
decoder_model = Model([decoder_inputs] + [encoder_h_state, encoder_c_state], [pd_decoder_softmax_outputs] + [pd_h_state, pd_c_state])

In [ ]:
input_stc = input()
re_line = re.sub('[#".!\n]', '', input_stc)
token_stc = re_line.split()
encode_stc = tokenizer_quest.texts_to_sequences([token_stc])
# 이제는 여러개 있는 게 아니기에 최대길이(maxlen)을 명시해줘야한다.
pad_stc = pad_sequences(encode_stc, maxlen=16, padding="post")

# 인코더의 마지막 시점의 셀/은닉 상태 값
states_value = encoder_model.predict(pad_stc)

# <start> 를 정수 인코딩해서 numpy array 로
predicted_seq = np.zeros((1,1)) # 출력해보면 [[0,]] 이다.
predicted_seq[0, 0] = ans_to_index['<start>']

# 각 시점마다 예측된 단어를 저장
decoded_stc = []

while True:
    output_words, h, c = decoder_model.predict([predicted_seq] + states_value)

    # print(np.argmax(output_words[0,0]))

    predicted_word = index_to_ans[np.argmax(output_words[0,0])]  

    if predicted_word == '<end>':
        break

    decoded_stc.append(predicted_word)

    # 처음에는 <start>, 지금은 예측된 단어가 있으니 이것을 인풋으로 넣어주기 위해서 변수의 값을 업데이트
    predicted_seq = np.zeros((1,1))
    predicted_seq[0, 0] = np.argmax(output_words[0,0])

    # 지금 시점의 상태 값을 다음 시점으로 넘기기 위해서 변수를 업데이트
    states_value = [h, c]

print(' '.join(decoded_stc))

요즘 유행하는 색깔이 뭐예요?	
그거는 제품은 제품은 제품은 안 나가요
